In [3]:
# -*- coding: utf-8 -*-

"""
Created on  April 29 15:30:00 2019

@author: grug in nanjing
"""
import os
import sys
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('agg') #deactivate matplotlib x11 backend #5
import pandas  as pd
import numpy  as np
import matplotlib.dates as dates
%matplotlib inline

In [4]:
def search(s, path=os.path.abspath('.')):  #os.path.abspath(path)：绝对路径

    for z in os.listdir(path):
        if os.path.isdir(path + os.path.sep + z):  # os.path.sep:路径分隔符 linux下就用这个了’/’
            #print('Currnet:', path)
            path2 = os.path.join(path, z) #；os.path.join(): 常用来链接路径
            #print('future:', path2)
            search(s, path2)
        elif os.path.isfile(path + os.path.sep + z): #检验给出的路径是否是一个文件：os.path.isfile()来自 <http://blog.csdn.net/devil_2009/article/details/7941241>
            if s in z:
                #print(os.path.join(path, z))
                filenames.append(os.path.join(path, z))
                #with open(path + os.path.sep + z, 'r') as fr:
                #    with open('save.txt', 'a') as fw:
                #        fw.write(path + '\t' + fr.read())

In [17]:
def plotData(ax,x,y,ptype,plabel):
    times=pd.to_datetime(x,format='%Y-%m-%d_%H:%M:%S')
    ax.xaxis.set_minor_locator(dates.DayLocator(interval=2))
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%m-%d'))
    ax.xaxis.set_major_locator(dates.MonthLocator())
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n'))

    ax.xaxis.grid(True, which="minor",color='lightgray', linestyle='-.', linewidth=0.2)
    ax.yaxis.grid(True, which="major",color='lightgray', linestyle='-.', linewidth=0.2)
    ax.tick_params(axis='both', which='major', labelsize=4)
    ax.tick_params(axis='both', which='minor', labelsize=4)
    ax.spines['top'].set_visible(False)  #去掉上边框
    ax.spines['right'].set_visible(False) #去掉右边框`
    xx=times.to_pydatetime()
    ax.plot(xx, y,ptype,label=plabel,alpha=0.6,markersize=1,linewidth=1) # 加上label参数添加图例 no color

In [15]:
def plot12station(value,base,base1,ytext,ptype,ptype1,plabel,plabel1):
    data=pd.read_table(base,sep='\s+',skiprows=1,names=["TIME","SITE","U10","V10","W","T2","SLP","RH2","RAIN",'PRE','CCN1','CCN1M','CCN2','CCN2M','CCN3','CCN3M','CCN4','CCN4M','CCN5','CCN5M','CCN6','CCN6M',"CLFRL","CLFRT","SWDN"])#数量不定的空格符可以用正则表达式\s+表示
    data1=pd.read_table(base1,sep='\s+',skiprows=1,names=["TIME","SITE","U10","V10","W","T2","SLP","RH2","RAIN",'PRE','CCN1','CCN1M','CCN2','CCN2M','CCN3','CCN3M','CCN4','CCN4M','CCN5','CCN5M','CCN6','CCN6M',"CLFRL","CLFRT","SWDN"])#数量不定的空格符可以用正则表达式\s+表示
    #print(data.head())
    #zhfont1 = matplotlib.font_manager.FontProperties(fname='/usr/share/fonts/wqy-microhei/wqy-microhei.ttc')
    data.set_index(['TIME','SITE'], inplace=True)
    data["T2"]=data["T2"]-273.15
    data1.set_index(['TIME','SITE'], inplace=True)
    data1["T2"]=data1["T2"]-273.15
    station= [
           u'Xuzhou',u'Huaiyin',u'Yancheng', u'Yangzhou',
           u'Taizhou',u'Nanjing', u'Zhenjiang', u'Changzhou',
           u'Shanghai',u'Wuxi',u'Nantong',u'Suzhou',
             ]
    plt.switch_backend('agg')#deactivate matplotlib x11 backend #5
    fig, ((ax1, ax2,ax3), (ax4, ax5,ax6),(ax7,ax8,ax9),(ax10, ax11,ax12))= plt.subplots(4, 3, sharex='col', sharey='row',figsize=(6,3))
    fig.subplots_adjust(left=0.1, bottom=0.08, right=0.9, top=0.96,hspace=0.3, wspace=0.2)
    #t="WRF Weather Forecast\nAtmospheric Environment Research Center\nNanjing University"
    #fig.suptitle(t,fontsize=6,x=0.53,y=1.1)
    axes=(ax1, ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12)
    for i in range(0,12):
        stat='SITE=="{stat_name}"'
        sta=stat.format(stat_name=station[i])
        print(sta)
        #print(data.index.levels[0])
        #print(data.query(sta)[value])
        plotData(axes[i],data.index.levels[0],data.query(sta)[value],ptype,plabel)#注意下站点对不对应
        plotData(axes[i],data1.index.levels[0],data1.query(sta)[value],ptype1,plabel1)#注意下站点对不对应
        #axes[i].set_title(station[i],fontsize=4,loc='left')
        axes[i].annotate(station[i], xy=(0.8, 1.0), xycoords="axes fraction",fontsize=6)
        plt.legend(bbox_to_anchor=(0.8,6.5), loc="lower left",
          borderaxespad=0, ncol=1)
    #fig.text(0.05, 0.5, r'%s'%ytext, va='center', rotation='vertical',fontsize=4,fontproperties=zhfont1)
    fig.text(0.5, -0.02, 'Time(BJT)', ha='center',fontsize=6)
    plt.savefig(value+'_qd03_new.png',dpi=300,bbox_inches='tight')
    plt.close()

In [18]:
if __name__ == "__main__":
    plot12station("T2","ccn_d03_mete.txt","feed_d03_mete.txt","2m Temperature (Du)",'r^-','k^-','aer_ra_feedback','no_ra_feedback')
    plot12station("RH2","ccn_d03_mete.txt","feed_d03_mete.txt","2m RH (%)",'gs-','ks-','aer_ra_feedback','no_ra_feedback')
    plot12station("SWDN","ccn_d03_mete.txt","feed_d03_mete.txt","SWDN (W/m^2)",'r^-','k^-','aer_ra_feedback','no_ra_feedback')
    plot12station("RAIN","ccn_d03_mete.txt","feed_d03_mete.txt","RAIN (mm/h)",'b.-','k.-','aer_ra_feedback','no_ra_feedback')
    plot12station("CLFRL","ccn_d03_mete.txt","feed_d03_mete.txt",u"CLFRL",'c2-','k2-','aer_ra_feedback','no_ra_feedback')
    plot12station("CLFRT","ccn_d03_mete.txt","feed_d03_mete.txt",u"CLFRT",'b2-','k2-','aer_ra_feedback','no_ra_feedback')

/data/c03n02/cliu/soft/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  
/data/c03n02/cliu/soft/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


SITE=="Xuzhou"
SITE=="Huaiyin"
SITE=="Yancheng"
SITE=="Yangzhou"
SITE=="Taizhou"
SITE=="Nanjing"
SITE=="Zhenjiang"
SITE=="Changzhou"
SITE=="Shanghai"
SITE=="Wuxi"
SITE=="Nantong"
SITE=="Suzhou"


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


SITE=="Xuzhou"
SITE=="Huaiyin"
SITE=="Yancheng"
SITE=="Yangzhou"
SITE=="Taizhou"
SITE=="Nanjing"
SITE=="Zhenjiang"
SITE=="Changzhou"
SITE=="Shanghai"
SITE=="Wuxi"
SITE=="Nantong"
SITE=="Suzhou"


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


...

SITE=="Xuzhou"
SITE=="Huaiyin"
SITE=="Yancheng"
SITE=="Yangzhou"
SITE=="Taizhou"
SITE=="Nanjing"
SITE=="Zhenjiang"
SITE=="Changzhou"
SITE=="Shanghai"
SITE=="Wuxi"
SITE=="Nantong"
SITE=="Suzhou"


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


SITE=="Xuzhou"
SITE=="Huaiyin"
SITE=="Yancheng"
SITE=="Yangzhou"
SITE=="Taizhou"
SITE=="Nanjing"
SITE=="Zhenjiang"
SITE=="Changzhou"
SITE=="Shanghai"
SITE=="Wuxi"
SITE=="Nantong"
SITE=="Suzhou"


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


SITE=="Xuzhou"
SITE=="Huaiyin"
SITE=="Yancheng"
SITE=="Yangzhou"
SITE=="Taizhou"
SITE=="Nanjing"
SITE=="Zhenjiang"
SITE=="Changzhou"
SITE=="Shanghai"
SITE=="Wuxi"
SITE=="Nantong"
SITE=="Suzhou"
